création de graphe sur un fichier

## Model using graph files not successful


In [25]:
import networkx as nx
from pathlib import Path
from sklearn.tree import DecisionTreeClassifier
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('all-MiniLM-L6-v2')
import json
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('all-MiniLM-L6-v2')

path_to_training = Path("training")
path_to_test = Path("test")

#######################################################################
def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]
training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
training_set.remove('IS1002a')
training_set.remove('IS1005d')
training_set.remove('TS3012c')

test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])

######################################################################
names=['Continuation', 'Explanation', 'Elaboration', 'Acknowledgement', 'Comment', 'Result', 'Narration','Question-answer_pair','Contrast','Clarification_question','Background','Alternation','Conditional','Q-Elab','Correction','Parallel']



graphes=[]
X_training = []
all_lens={}
for transcription_id in training_set:
    X_training_part=[]
    l=0
    with open(path_to_training / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)
    
    for utterance in transcription:
        X_training_part.append(utterance["speaker"] + ": " + utterance["text"])
        l+=1
    all_lens[transcription_id]  = l  # On connait maintenant la taille du graphe
    X_training.extend(X_training_part)
    
    G=nx.Graph()
    
    for index, phrase in enumerate(X_training_part):
        G.add_node(index, text=phrase)
    edges=[]
    with open(path_to_training / f"{transcription_id}.txt", "r") as file:
        transcription = file.read().splitlines()
        for utterance in transcription:
            splitted = utterance.split()
            a,b=int(splitted[0]),int(splitted[2])
            #c,d=X_training_part[a],X_training_part[b]
            edges.append((a,b))
    G.add_edges_from(edges)  
    graphes.append(G)      
            







     



In [28]:
"""""
for n, attributs in graphes[0].nodes(data=True):
    print(f"Nœud {n}: {attributs}") """

'""\nfor n, attributs in graphes[0].nodes(data=True):\n    print(f"Nœud {n}: {attributs}") '

In [29]:
"""""
# Supposons que 'graphes[0]' est votre graphe
for edge in graphes[0].edges(data=True):
    node1, node2, attributs = edge
    print(f"Arête entre les nœuds {node1} et {node2} : {attributs}")

    poids = attributs.get('poids', None)
    if poids is not None:
        print(f"Poids de l'arête : {poids}")

"""""

'""\n# Supposons que \'graphes[0]\' est votre graphe\nfor edge in graphes[0].edges(data=True):\n    node1, node2, attributs = edge\n    print(f"Arête entre les nœuds {node1} et {node2} : {attributs}")\n\n    poids = attributs.get(\'poids\', None)\n    if poids is not None:\n        print(f"Poids de l\'arête : {poids}")\n\n'

In [30]:
graphe_fusionne = nx.Graph()
for g in graphes:
    graphe_fusionne = nx.compose(graphe_fusionne, g)

In [34]:
y_training = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)
for transcription_id in training_set:
    y_training+=training_labels[transcription_id]
